In [9]:
import numpy as np
import json

res_path = "/Volumes/Isis/Thesis/qcd-experiments/results/pearson/front/config2/dabbad00/aggregate/res_dummy.json"

with open(res_path, "r") as jsonfile:
    res = json.load(jsonfile)

tx = res['TX']
rx = res['RX']

count = 0
c_rep = 0
for (s_id, data) in tx.items():
    c = data['count']
    c_rep += c
    print(f'{s_id}: {c} reps')
    count += 1

ratio_samples = count/315.0
samp_succ = c_rep/(count*20.0)
overall_succ = c_rep/(315.0*20.0)
print(f'\ntotal samples: {count} ({ratio_samples*100:.2f}%)')
print(f'total successful results: {c_rep} ({samp_succ*100:.2f}%) ({overall_succ*100:.2f}%)')

246: 20 reps
107: 20 reps
133: 20 reps
206: 20 reps
002: 20 reps
040: 20 reps
022: 20 reps
127: 20 reps
098: 20 reps
029: 20 reps
077: 20 reps
225: 20 reps
021: 20 reps
026: 20 reps
012: 20 reps
008: 10 reps
148: 20 reps
053: 20 reps
006: 20 reps
067: 20 reps
142: 20 reps
065: 20 reps
088: 20 reps
100: 20 reps
036: 20 reps
286: 20 reps
102: 20 reps
129: 20 reps
094: 20 reps
184: 20 reps
123: 20 reps
229: 20 reps
057: 20 reps
025: 20 reps
004: 20 reps
177: 20 reps
039: 20 reps
033: 20 reps
260: 20 reps
312: 20 reps
099: 20 reps
154: 20 reps
153: 20 reps
096: 20 reps
201: 20 reps
072: 20 reps
017: 20 reps
157: 20 reps
071: 20 reps
062: 20 reps
056: 20 reps
122: 20 reps
150: 20 reps
007: 17 reps
014: 20 reps
035: 20 reps
037: 20 reps
066: 19 reps
054: 20 reps
003: 20 reps
075: 20 reps
109: 20 reps
120: 20 reps
064: 20 reps
111: 20 reps
307: 20 reps
055: 20 reps
141: 20 reps
005: 20 reps
024: 20 reps
034: 20 reps
001: 20 reps
087: 20 reps
020: 20 reps
009: 20 reps
038: 20 reps
236: 20 reps

In [10]:
treshold = 0.738
for (s_id, data) in rx.items():
    m = data['mean']
    if m < treshold:
        print(f'Outlier {s_id} with mean {m}')

Outlier 098 with mean 0.39251618073299294
Outlier 077 with mean 0.17852557514331965
Outlier 012 with mean 0.621738334878828
Outlier 008 with mean 0.6773566267827242
Outlier 148 with mean 0.6073249999988924
Outlier 184 with mean 0.7236993400112589
Outlier 229 with mean 0.32288835324139514
Outlier 017 with mean 0.632554483631589
Outlier 007 with mean 0.7016715053981082
Outlier 066 with mean 0.5172859829062458
Outlier 307 with mean 0.7377929490516666
Outlier 055 with mean 0.476265660588748


# Investigate outliers

In [27]:
%%capture
from matplotlib import pyplot as plt

resdir = "/Volumes/Isis/Thesis/qcd-experiments/results"
confs = ["config1", "config2", "config3", "config4"]

conf1_cafe_f = f"{resdir}/pearson/front/config1/baddcafe/aggregate/res_dummy.json"
conf2_cafe_f = f"{resdir}/pearson/front/config2/baddcafe/aggregate/res_dummy.json"
conf3_cafe_f = f"{resdir}/pearson/front/config3/baddcafe/aggregate/res_dummy.json"
conf4_cafe_f = f"{resdir}/pearson/front/config4/baddcafe/aggregate/res_dummy.json"
cafe_f = [conf1_cafe_f, conf2_cafe_f, conf3_cafe_f, conf4_cafe_f]

conf1_dead_f = f"{resdir}/pearson/front/config1/dead/aggregate/res_dummy.json"
conf2_dead_f = f"{resdir}/pearson/front/config2/dead/aggregate/res_dummy.json"
conf3_dead_f = f"{resdir}/pearson/front/config3/dead/aggregate/res_dummy.json"
conf4_dead_f = f"{resdir}/pearson/front/config4/dead/aggregate/res_dummy.json"
dead_f = [conf1_dead_f, conf2_dead_f, conf3_dead_f, conf4_dead_f]

conf1_dabba_f = f"{resdir}/pearson/front/config1/dabbad00/aggregate/res_dummy.json"
conf2_dabba_f = f"{resdir}/pearson/front/config2/dabbad00/aggregate/res_dummy.json"
conf3_dabba_f = f"{resdir}/pearson/front/config3/dabbad00/aggregate/res_dummy.json"
conf4_dabba_f = f"{resdir}/pearson/front/config4/dabbad00/aggregate/res_dummy.json"
dabba_f = [conf1_dabba_f, conf2_dabba_f, conf3_dabba_f, conf4_dabba_f]

def aggregate_and_means(data):
    means = np.array([])
    for (s_id, reps) in data.items():
        reps = reps['reps']
        avg = np.nanmean(list(reps.values()))
        means = np.append(means, avg)
        # count how many non-nan values
        r_count = (~np.isnan(list(reps.values()))).sum()
        data[s_id] = {'count': int(r_count), 'mean': avg, 'reps': reps}

    return means, data

cafe = {}
data_cafe = []
for file, conf in zip(cafe_f, confs):
    with open(file, "r") as f:
        jsonfile = json.load(f)
    pearsons_RX = jsonfile['RX']
    (rx_means, pearsons_RX) = aggregate_and_means(pearsons_RX)
    cafe[conf] = pearsons_RX
    data_cafe += [rx_means]

dead = {}
data_dead = []
for file, conf in zip(dead_f, confs):
    with open(file, "r") as f:
        jsonfile = json.load(f)
    pearsons_RX = jsonfile['RX']
    (rx_means, pearsons_RX) = aggregate_and_means(pearsons_RX)
    dead[conf] = pearsons_RX
    data_dead += [rx_means]

dabba = {}
data_dabba = []
for file, conf in zip(dabba_f, confs):
    with open(file, "r") as f:
        jsonfile = json.load(f)
    pearsons_RX = jsonfile['RX']
    (rx_means, pearsons_RX) = aggregate_and_means(pearsons_RX)
    dabba[conf] = pearsons_RX
    data_dabba += [rx_means]

bpcafe = plt.boxplot(data_cafe);
bpdead = plt.boxplot(data_dead);
bpdabba = plt.boxplot(data_dabba);

cafe_maxf = {}
outliers = [flier.get_ydata() for flier in bpcafe['fliers']]
for i, conf in enumerate(confs):
    maxf = max(outliers[i]) if len(outliers[i]>0) else -100000
    cafe_maxf[conf] = maxf
dead_maxf = {}
outliers = [flier.get_ydata() for flier in bpdead['fliers']]
for i, conf in enumerate(confs):
    maxf = max(outliers[i]) if len(outliers[i]>0) else -100000
    dead_maxf[conf] = maxf
dabba_maxf = {}
outliers = [flier.get_ydata() for flier in bpdabba['fliers']]
for i, conf in enumerate(confs):
    maxf = max(outliers[i]) if len(outliers[i]>0) else -100000
    dabba_maxf[conf] = maxf



In [37]:

treshold = cafe_maxf['config3']
rx = cafe['config3']
cafe_flier_id = []
for (s_id, data) in rx.items():
    m = data['mean']
    if m <= treshold:
        cafe_flier_id += [int(s_id)]
        print(f'Outlier {s_id} with mean {m}')
cafe_flier_id.sort()

print("----------------")
treshold = dead_maxf['config3']
rx = dead['config3']
dead_flier_id = []
for (s_id, data) in rx.items():
    m = data['mean']
    if m <= treshold:
        dead_flier_id += [int(s_id)]
        print(f'Outlier {s_id} with mean {m}')
dead_flier_id.sort()

print("----------------")
treshold = dabba_maxf['config3']
rx = dabba['config3']
dabba_flier_id = []
for (s_id, data) in rx.items():
    m = data['mean']
    if m <= treshold:
        dabba_flier_id += [int(s_id)]
        print(f'Outlier {s_id} with mean {m}')
dabba_flier_id.sort()

print(cafe_flier_id)
print(dead_flier_id)
print(dabba_flier_id)

Outlier 012 with mean -0.2908392484098171
Outlier 148 with mean -0.5275827037615857
Outlier 184 with mean -0.2978820244382193
Outlier 229 with mean -0.5000947128366369
Outlier 007 with mean -0.29866483631792695
Outlier 066 with mean -0.33612849835015
Outlier 307 with mean -0.2608826828037017
Outlier 015 with mean -0.30337389801407494
----------------
----------------
Outlier 098 with mean 0.0496615245989875
Outlier 077 with mean -0.0933006679068947
Outlier 012 with mean -0.09155696111154059
Outlier 148 with mean 0.1610438122923292
Outlier 229 with mean 0.025550083684503743
Outlier 033 with mean 0.29841111336843273
Outlier 007 with mean 0.3189602660530857
Outlier 066 with mean 0.3191151100469954
Outlier 055 with mean 0.4815025262182829
[7, 12, 15, 66, 148, 184, 229, 307]
[]
[7, 12, 33, 55, 66, 77, 98, 148, 229]
